In [1]:
import pandas as pd

In [13]:
data=pd.read_excel('usyd_data_output.xlsx')
data.head()



,Name,% Docs Cited,Web of Science Documents,Rank,Times Cited,Affiliation,Category Normalized Citation Impact,Citation Impact,Average Percentile,Journal Normalized Citation Impact,H-Index,Impact Relative to World,Citations From Patents,% Documents in Top 1%,% Documents in Top 10%,% Highly Cited Papers,Lastname,Firstname,hcr_label
0,"Holmes, Edward C.",94.71,208,1.0,25832,University of Sydney,7.370518,124.192308,71.973397,2.000000,45,15.638232,245,9.62,34.62,7.69,Holmes,Edward C.,1
1,"Islam, Sheikh Mohammed Shariful",93.85,65,2.0,24791,University of Sydney,33.078962,381.400000,83.171746,2.897538,37,48.025694,0,40.00,69.23,32.31,Islam,Sheikh Mohammed Shariful,0
2,"Tao, Dacheng",86.99,369,3.0,15473,University of Sydney,5.139136,41.932249,71.152365,1.750461,59,5.280088,52,12.47,45.53,4.61,Tao,Dacheng,2
3,"George, Jacob",68.91,267,4.0,13179,University of Sydney,4.372782,49.359551,53.223301,1.693933,46,6.215330,6,9.74,31.46,8.99,George,Jacob,1
4,"Woodward, Mark",87.36,269,5.0,12241,University of Sydney,4.379590,45.505576,65.052643,1.372900,44,5.730039,2,5.58,34.57,3.35,Woodward,Mark,0


In [18]:
###Recursive Feature Elimination, RFE
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

data = pd.read_excel('usyd_data_output.xlsx')

data['hcr_label'] = data['hcr_label'].apply(lambda x: 1 if x != 0 else 0)

X = data.drop(columns=['hcr_label', 'Name', 'Lastname', 'Firstname', 'Affiliation', 'Rank'])  # 删除不相关特征
y = data['hcr_label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rf = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=10) 
rfe.fit(X_scaled, y)

selected_features = X.columns[rfe.support_]
print("Selected Features: ", selected_features)

selected_importances = rfe.estimator_.feature_importances_

selected_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': selected_importances
}).sort_values(by='Importance', ascending=False)

ranking = rfe.ranking_

ranking_df = pd.DataFrame({
    'Feature': X.columns,
    'Ranking': ranking
}).sort_values(by='Ranking')

print("Selected Features Importance: \n", selected_df)
print("Feature Ranking: \n", ranking_df)


Selected Features:  Index(['% Docs Cited', 'Web of Science Documents', 'Times Cited',
       'Category Normalized Citation Impact', 'Citation Impact',
       'Average Percentile', 'H-Index', 'Impact Relative to World',
       '% Documents in Top 1%', '% Highly Cited Papers'],
      dtype='object')
Selected Features Importance: 
                                Feature  Importance
2                          Times Cited    0.160230
1             Web of Science Documents    0.153950
6                              H-Index    0.133339
3  Category Normalized Citation Impact    0.088958
5                   Average Percentile    0.083344
9                % Highly Cited Papers    0.078829
7             Impact Relative to World    0.078191
0                         % Docs Cited    0.075532
8                % Documents in Top 1%    0.074626
4                      Citation Impact    0.073000
Feature Ranking: 
                                 Feature  Ranking
0                          % Docs Cited 

In [19]:
### Random Forest Feature Importance

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

data = pd.read_excel('usyd_data_output.xlsx')

data['hcr_label'] = data['hcr_label'].apply(lambda x: 1 if x != 0 else 0)

X = data.drop(columns=['hcr_label', 'Name', 'Lastname', 'Firstname', 'Affiliation', 'Rank'])  # 删除不相关特征
y = data['hcr_label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_scaled, y)

importances = rf.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importance: \n", importance_df)



Feature Importance: 
                                 Feature  Importance
7                               H-Index    0.136907
2                           Times Cited    0.134755
1              Web of Science Documents    0.124949
3   Category Normalized Citation Impact    0.084051
5                    Average Percentile    0.073736
12                % Highly Cited Papers    0.071236
4                       Citation Impact    0.060741
10                % Documents in Top 1%    0.060583
8              Impact Relative to World    0.060537
0                          % Docs Cited    0.054166
6    Journal Normalized Citation Impact    0.053230
9                Citations From Patents    0.046861
11               % Documents in Top 10%    0.038249


In [21]:
###Principal Component Analysis, PCA
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm

data = pd.read_excel('usyd_data_output.xlsx')

data['hcr_label'] = data['hcr_label'].apply(lambda x: 1 if x != 0 else 0)

X = data.drop(columns=['hcr_label', 'Name', 'Lastname', 'Firstname', 'Affiliation', 'Rank'])  # 删除不相关特征
y = data['hcr_label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=10)  
X_pca = pca.fit_transform(X_scaled)

explained_variance = pca.explained_variance_ratio_

components = pd.DataFrame(pca.components_, columns=X.columns)
components_abs = components.abs()
components_abs.columns = X.columns

feature_importance = components_abs.sum(axis=0)

X_with_const = sm.add_constant(X_scaled)  
model = sm.OLS(y, X_with_const)
results = model.fit()

p_values = results.pvalues[1:] 

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'PCA Importance': feature_importance,
    'p-value': p_values.values
}).sort_values(by='PCA Importance', ascending=False)

print("Feature Importance and p-value: \n", importance_df)



Feature Importance and p-value: 
                                                                  Feature  \
% Documents in Top 1%                              % Documents in Top 1%   
Journal Normalized Citation Impact    Journal Normalized Citation Impact   
% Highly Cited Papers                              % Highly Cited Papers   
Category Normalized Citation Impact  Category Normalized Citation Impact   
Citations From Patents                            Citations From Patents   
Times Cited                                                  Times Cited   
% Documents in Top 10%                            % Documents in Top 10%   
Citation Impact                                          Citation Impact   
Impact Relative to World                        Impact Relative to World   
% Docs Cited                                                % Docs Cited   
Web of Science Documents                        Web of Science Documents   
H-Index                                               

In [23]:
### Lasso
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

data = pd.read_excel('usyd_data_output.xlsx')

data['hcr_label'] = data['hcr_label'].apply(lambda x: 1 if x != 0 else 0)

X = data.drop(columns=['hcr_label', 'Name', 'Lastname', 'Firstname', 'Affiliation', 'Rank'])  # 删除不相关特征
y = data['hcr_label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lasso = LassoCV(cv=5, random_state=42).fit(X_scaled, y)

lasso_importance = np.abs(lasso.coef_)

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso Importance': lasso_importance
}).sort_values(by='Lasso Importance', ascending=False)

print("Feature Importance from Lasso: \n", importance_df)


Feature Importance from Lasso: 
                                 Feature  Lasso Importance
2                           Times Cited          0.003228
0                          % Docs Cited          0.000000
1              Web of Science Documents          0.000000
3   Category Normalized Citation Impact          0.000000
4                       Citation Impact          0.000000
5                    Average Percentile          0.000000
6    Journal Normalized Citation Impact          0.000000
7                               H-Index          0.000000
8              Impact Relative to World          0.000000
9                Citations From Patents          0.000000
10                % Documents in Top 1%          0.000000
11               % Documents in Top 10%          0.000000
12                % Highly Cited Papers          0.000000


/Users/ericma/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ericma/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ericma/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ericma/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge

In [25]:
#### Chi-squared Test

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2, SelectKBest

data = pd.read_excel('usyd_data_output.xlsx')

data['hcr_label'] = data['hcr_label'].apply(lambda x: 1 if x != 0 else 0)

X = data.drop(columns=['hcr_label', 'Name', 'Lastname', 'Firstname', 'Affiliation', 'Rank'])  
y = data['hcr_label']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

chi2_scores, p_values = chi2(X_scaled, y)

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Chi2 Score': chi2_scores,
    'p-value': p_values
}).sort_values(by='Chi2 Score', ascending=False)

print("Feature Importance and p-value from Chi-squared Test: \n", importance_df)



Feature Importance and p-value from Chi-squared Test: 
                                 Feature  Chi2 Score       p-value
2                           Times Cited  422.588268  6.669345e-94
9                Citations From Patents  304.324365  3.764090e-68
1              Web of Science Documents  225.917082  4.632261e-51
7                               H-Index  165.222935  8.177310e-38
11               % Documents in Top 10%   16.173433  5.779909e-05
12                % Highly Cited Papers   13.232862  2.750834e-04
10                % Documents in Top 1%    7.890029  4.970807e-03
5                    Average Percentile    4.169961  4.114674e-02
0                          % Docs Cited    1.436163  2.307613e-01
8              Impact Relative to World    0.838594  3.597992e-01
4                       Citation Impact    0.838594  3.597992e-01
3   Category Normalized Citation Impact    0.388344  5.331709e-01
6    Journal Normalized Citation Impact    0.050388  8.223897e-01
